In [113]:
!pip install torch 

In [114]:
!pip install nltk

In [115]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score, precision_score, f1_score

import torch 
from torch import nn
from torch import optim

import datetime

In [116]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import ngrams
from nltk import word_tokenize, pos_tag
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys

In [117]:
import nltk

nltk.download("wordnet")
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [118]:
# Load the training and validation data

train = pd.read_csv('../public_data/train/track_a/eng.csv')
val = pd.read_csv('../public_data/dev/track_a/eng_a.csv')

train.head()

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0


In [131]:
## Add Tokeniser search
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# SPACY TOKENISER SETUP 
# import spacy
# nlp = spacy.load("en_core_web_sm")

# NLTK TREEBANK TOKENISER SETUP
# from nltk.tokenize import TreebankWordTokenizer

# NLTK WORDPUNCT TOKENISER SETUP 
# from nltk.tokenize import WordPunctTokenizer


# Tokenize and encode the text

def pre_process(text, config):
    """ 
    Performs Different preprocessing operations.

    Parameters:
    text (string): passes a line of text (assume sentence segmentation has already been done)

    Returns:
    List[string]: Should return a list of tokens.
    """

    def separate_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", text)
        return text

    def remove_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\2", text)
        return text
        
    def tokenize_text(text):
        # Original tokeniser:
        # tokens = re.split(r"\s+",text)
        # tokens = [t.lower() for t in tokens]
        
        # BERT TOKENISER SETUP
        # Encode text returns a BatchEncoding object
        encoded_input = tokenizer(text, return_tensors='pt', add_special_tokens=True)
        # Convert token ids to words
        tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
        return tokens

        # SPACY TOKENISER SETUP
        # doc = nlp(text)
        # # Collect only the text of each token in the document
        # tokens = [token.text for token in doc]

        # NLTK TREEBANKWORD TOKENIZER 
        # tokenizer = TreebankWordTokenizer()
        # tokens = tokenizer.tokenize(text)

        # NLTK WORDPUNCT TOKENIZER
        # tokenizer = WordPunctTokenizer()
        # tokens = tokenizer.tokenize(text)

        # Return the list of tokens
        # return tokens


    def apply_stemming(tokens):
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return stemmed_tokens

    def apply_lemmatization(tokens):
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return lemmatized_tokens

    def generate_ngrams_from_tokens(tokens, n):
        return list(ngrams(tokens, n))

    # Separate Punctuation otherwise Remove it
    
    if config["sep_pn"] and not config["rm_pn"]:
        text = separate_punctuation(text)

    if config["rm_pn"] and not config["sep_pn"]:
        text = remove_punctuation(text)
    
    # tokenize text
    
    tokens = tokenize_text(text)

    # Apply Lemmatization or Stemming

    if config["apply_stemming"]:
        tokens = apply_stemming(tokens)
    if config["apply_lemmatization"]:
        tokens = apply_lemmatization(tokens)

    # Generate bigrams, trigrams and quadgrams
    if config["add_bigrams"]:
        bigrams = generate_ngrams_from_tokens(tokens, 2)
        bg = [i + " " + j for (i,j) in bigrams]
        tokens += bg

    # Remove Stop words
    
    if config["rm_sw"]:
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if w not in stop_words]

    # stop_words = set(stopwords.words('english'))
    # tokens = [w for w in tokens if w not in stop_words]

    return " ".join(tokens)

In [120]:
# import itertools

# def grid_configurations():
#     options = [
#         "sep_pn", "rm_pn", "apply_lemmatization", "apply_stemming", "add_bigrams", "rm_sw"
#     ]
    
#     # Generate all combinations of True/False for each option
#     combinations = itertools.product([True, False], repeat=len(options))
    
#     configurations = []
    
#     # Create a dictionary for each combination
#     for combo in combinations:
#         config = {options[i]: combo[i] for i in range(len(options))}
#         configurations.append(config)
    
#     return configurations

# # Example usage
# results = []
# configs = grid_configurations()
# for _i, config in enumerate(configs):
#     vectorizer = CountVectorizer()
#     X_train = vectorizer.fit_transform([pre_process(i, config) for i in train["text"]]).toarray()
#     X_val = vectorizer.transform(val['text'].str.lower()).toarray()

#     emotions = ['Joy','Sadness','Surprise','Fear','Anger']
#     y_train = train[emotions].values
#     y_val = val[emotions].values

#     X_train_t = torch.Tensor(X_train)
#     y_train_t = torch.Tensor(y_train)

#     X_val_t = torch.Tensor(X_val)
#     y_val_t = torch.Tensor(y_val)

#     model = nn.Sequential(
#           nn.Linear(X_train.shape[1], 100),
#           nn.ReLU(),
#           nn.Dropout(0.3),
#           nn.Linear(100, y_train.shape[1])
#         )
    
#     criterion = nn.BCEWithLogitsLoss()
#     optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)
#     loss = None
#     # Train for a set number of epochs
#     for epoch in range(200):
#         optimizer.zero_grad()
#         output = model(X_train_t)
#         loss = criterion(output, y_train_t)
#         loss.backward()
#         optimizer.step()
#     results.append((round(loss.item(),3), _i))
#     print(_i, round(loss.item(),3), config)
# print("MIN LOSS: ", min(results))

In [129]:
# [(i, pre_process(i, {'sep_pn': True, 'rm_pn': False, 'apply_lemmatization': True, 'apply_stemming': True, 'add_bigrams': True, 'rm_sw': False})) for i in train["text"]]

In [122]:
# ## Add Tokeniser search
# from transformers import BertTokenizer

# # Load pre-trained BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize and encode the text

# tokens = tokenizer("Example text for sentiment analysis", return_tensors="pt")

# `inputs` now contains input IDs and attention masks that can be fed into a BERT model

In [123]:
# import spacy
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("Your sample text goes here.")
# tokens = [token.text for token in doc]

# from nltk.tokenize import TweetTokenizer
# tokenizer = TweetTokenizer()
# tokens = tokenizer.tokenize("Your sample tweet goes here :) #exciting")

In [ ]:
vectorizer = CountVectorizer()
# Optimal config #18 
config = {'sep_pn': True, 'rm_pn': False, 'apply_lemmatization': True, 'apply_stemming': True, 'add_bigrams': True, 'rm_sw': False}
X_train = vectorizer.fit_transform([pre_process(i, config) for i in train["text"]]).toarray()
X_val = vectorizer.transform(val['text'].str.lower()).toarray()

emotions = ['Joy','Sadness','Surprise','Fear','Anger']
y_train = train[emotions].values
y_val = val[emotions].values

print(val)

                        id                                               text  \
0    eng_dev_track_a_00001            My mouth fell open `` No, no, no... I..   
1    eng_dev_track_a_00002  You can barely make out your daughter's pale f...   
2    eng_dev_track_a_00003  But after blinking my eyes for a few times lep...   
3    eng_dev_track_a_00004  Slowly rising to my feet I came to the conclus...   
4    eng_dev_track_a_00005  I noticed this months after moving in and doin...   
..                     ...                                                ...   
111  eng_dev_track_a_00112                       "ARcH stop your progression.   
112  eng_dev_track_a_00113        This 'star', starts to move across the sky.   
113  eng_dev_track_a_00114                                  and my feet hurt.   
114  eng_dev_track_a_00115        so i cried my eyes out and did the drawing.   
115  eng_dev_track_a_00116                              They were coal black.   

     Anger  Fear  Joy  Sadn

In [133]:
X_train_t = torch.Tensor(X_train)
y_train_t = torch.Tensor(y_train)

X_val_t = torch.Tensor(X_val)
y_val_t = torch.Tensor(y_val)

In [134]:
print(f'Shape of X: {X_train.shape}')
print(f'Shape of y: {y_train.shape}')
print(f'Number of positives per emotion class:')
_ = [print(f' - {e}: {v} ({round(100*v/len(y_train))}%)') for e,v in zip(emotions, y_train.sum(axis=0))]

Shape of X: (2768, 4074)
Shape of y: (2768, 5)
Number of positives per emotion class:
 - Joy: 674 (24%)
 - Sadness: 878 (32%)
 - Surprise: 839 (30%)
 - Fear: 1611 (58%)
 - Anger: 333 (12%)


In [135]:
model = nn.Sequential(
          nn.Linear(X_train.shape[1], 100),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

In [136]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

In [137]:
# Train for a set number of epochs
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

Epoch 0: Loss: 0.701
Epoch 100: Loss: 0.565
Epoch 200: Loss: 0.552
Epoch 300: Loss: 0.543
Epoch 400: Loss: 0.536
Epoch 500: Loss: 0.53
Epoch 600: Loss: 0.522
Epoch 700: Loss: 0.516
Epoch 800: Loss: 0.507
Epoch 900: Loss: 0.499


In [138]:
def get_predictions(X_val, model, threshold=0.5):
    sig = nn.Sigmoid() 
    yhat = sig(model(X_val)).detach().numpy()
    y_pred = yhat > threshold
    
    return y_pred

In [139]:
y_pred = get_predictions(X_val_t, model, 0.45)
# print(y_pred)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

val_data_with_pred = val_data_with_pred.astype(int)

val_data_with_pred['id'] = val['id']

val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

print(val_data_with_pred)

                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      0     1    1        1         0
1    eng_dev_track_a_00002      0     0    1        1         0
2    eng_dev_track_a_00003      1     0    0        1         0
3    eng_dev_track_a_00004      0     0    0        1         0
4    eng_dev_track_a_00005      0     1    0        1         0
..                     ...    ...   ...  ...      ...       ...
111  eng_dev_track_a_00112      0     0    1        1         0
112  eng_dev_track_a_00113      0     0    1        1         0
113  eng_dev_track_a_00114      0     1    0        1         0
114  eng_dev_track_a_00115      0     0    0        1         0
115  eng_dev_track_a_00116      0     1    1        1         0

[116 rows x 6 columns]


In [ ]:
# def evaluate(y_val, y_pred):
#     for average in ['micro', 'macro']:
#         recall = recall_score(y_val, y_pred, average=average, zero_division=0)
#         precision = precision_score(y_val, y_pred, average=average, zero_division=0)
#         f1 = f1_score(y_val, y_pred, average=average, zero_division=0)
    
#         print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

In [ ]:
# evaluate(y_val, y_pred) EVALS WON'T WORK HERE 

In [ ]:
# def evaluate_per_class(y_val, y_pred):
#     for i, emotion in enumerate(emotions):
#         print(f'*** {emotion} ***')
    
#         recall = recall_score(y_val[:,i], y_pred[:,i], zero_division=0)
#         precision = precision_score(y_val[:,i], y_pred[:,i], zero_division=0)
#         f1 = f1_score(y_val[:,i], y_pred[:,i], zero_division=0)
        
#         print(f'recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}\n')

In [ ]:
# evaluate_per_class(y_val, y_pred) EVALS WON'T WORK HERE 

In [ ]:
# weights = y_train.sum(axis=0)/y_train.sum()
# weights = max(weights)/weights

In [ ]:
# # Define model 
# model = nn.Sequential(
#           nn.Linear(X_train.shape[1], 100),
#           nn.ReLU(),
#           nn.Dropout(0.3),
#           nn.Linear(100, y_train.shape[1])
#         )

# Define training parameters
# criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(weights)) # <-- weights assigned to optimiser
# optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

# # Train for a number of epochs
# for epoch in range(1000):
#     optimizer.zero_grad()
#     output = model(X_train_t)
#     loss = criterion(output, y_train_t)
#     loss.backward()
#     optimizer.step()
#     if epoch % 100 == 0:
#         print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

# # Get predictions
# y_pred = get_predictions(X_val_t, model, 0.45)

# # Create a DataFrame to save to CSV
# val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# # val_data_with_pred['True_Label'] = y_test
# # val_data_with_pred['Predictions'] = dummy_predictions

# val_data_with_pred = val_data_with_pred.astype(int)

# val_data_with_pred['id'] = val['id']

# val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

# print(val_data_with_pred)

# # Evaluate: EVALS WON'T WORK HERE 
# # print('\n\nEVALUATION\n')
# # evaluate(y_val, y_pred)

# # print('\nPER CLASS BREAKDOWN\n')
# # evaluate_per_class(y_val, y_pred)

Epoch 0: Loss: 0.882
Epoch 100: Loss: 0.834
Epoch 200: Loss: 0.803
Epoch 300: Loss: 0.765
Epoch 400: Loss: 0.723
Epoch 500: Loss: 0.676
Epoch 600: Loss: 0.63
Epoch 700: Loss: 0.59
Epoch 800: Loss: 0.551
Epoch 900: Loss: 0.518
                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      0     1    1        1         0
1    eng_dev_track_a_00002      0     0    1        1         0
2    eng_dev_track_a_00003      1     0    0        1         0
3    eng_dev_track_a_00004      0     1    0        1         0
4    eng_dev_track_a_00005      0     0    1        1         0
..                     ...    ...   ...  ...      ...       ...
111  eng_dev_track_a_00112      0     0    1        1         0
112  eng_dev_track_a_00113      0     0    1        1         0
113  eng_dev_track_a_00114      0     1    0        1         0
114  eng_dev_track_a_00115      0     0    1        1         0
115  eng_dev_track_a_00116      0     0    1        1         0

[116 

In [ ]:
# val_data_with_pred

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,0,0,1,0
1,eng_dev_track_a_00002,0,0,1,1,0
2,eng_dev_track_a_00003,0,0,0,1,0
3,eng_dev_track_a_00004,0,0,0,1,0
4,eng_dev_track_a_00005,0,0,0,1,0
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,0,0,1,0
112,eng_dev_track_a_00113,0,1,1,1,0
113,eng_dev_track_a_00114,0,0,0,1,0
114,eng_dev_track_a_00115,0,0,0,1,0
